In [23]:
import re

def remove_whitespace_inside_angle_brackets(s):
    def repl(match):
        return '<' + ''.join(match.group(1).split()) + '>'
    
    return re.sub(r'<(.*?)>', repl, s)

# Test
s = "a b c:l<d, e, f> g o"
print(remove_whitespace_inside_angle_brackets(s))  # Expected: "a b c:l<d,e,f> g o"
s = "d, e, c:l<d, e, f> g o"
print(remove_whitespace_inside_angle_brackets(s))  # Expected: "a b c:l<d,e,f> g o"

a b c:l<d,e,f> g o
d, e, c:l<d,e,f> g o


In [34]:
def save_current_pattern(patterns, current_name, current_pattern):
    if current_name and current_pattern:
        patterns[current_name] = ' '.join(current_pattern)

def parse_declarations(pattern_declarations):
    patterns = {}
    lines = pattern_declarations.strip().split('\n')
    current_name = None
    current_pattern = []

    for line in lines:
        stripped_line = line.strip()
        
        # Check if line defines a new pattern
        if ' ' in stripped_line and stripped_line.split(' ', 1)[0] == "pat":
            save_current_pattern(patterns, current_name, current_pattern)
            current_pattern = []

            _, current_name, pattern = stripped_line.split(' ', 2)

            if "<" in current_name:
                current_name, type_params = current_name.split("<", 1)
                type_params = remove_whitespace_inside_angle_brackets("<" + type_params)
                current_pattern.append(type_params)

            current_pattern.append(pattern)
        else:
            current_pattern.append(stripped_line)

    save_current_pattern(patterns, current_name, current_pattern)
    return patterns


pattern_declarations = """
pat rectangle width:float mm x height:float mm
pat box base:rectangle x depth:float mm
pat krabice width:float mm x
                  height:float mm x
                  depth:float mm
pat choice C weight:int || B foo:int || A bar:int
pat int_list item:int rest_of_list:int_list || end of list
pat list<a> item:a rest_of_list:list<a> || end of list
pat double_recursion_a drb:double_recursion_b dra:double_recursion_a
pat double_recursion_b dra:double_recursion_a
"""

patterns = parse_declarations(pattern_declarations)
patterns



{'rectangle': 'width:float mm x height:float mm',
 'box': 'base:rectangle x depth:float mm',
 'krabice': 'width:float mm x height:float mm x depth:float mm',
 'choice': 'C weight:int || B foo:int || A bar:int',
 'int_list': 'item:int rest_of_list:int_list || end of list',
 'list': '<a> item:a rest_of_list:list<a> || end of list',
 'double_recursion_a': 'drb:double_recursion_b dra:double_recursion_a',
 'double_recursion_b': 'dra:double_recursion_a'}

In [40]:
def resolve_type(htype, patterns, recursed_types):
    if htype in patterns:
        if htype not in recursed_types:
            return resolve_pattern(htype, patterns[htype], patterns, recursed_types + [htype])
        else:
            return [f"{htype}:{htype}"], [htype]
    return [f"name:{htype}"], []

def resolve_pattern(pattern_name, pattern, patterns, recursed_types=None):
    if recursed_types is None:
        recursed_types = [pattern_name]
        
    words = pattern.split()
    resolved_words = []
    recursed_patterns = []
    
    for word in words:
        if ":" in word:
            hname, htype = word.split(":")
            resolved_word, recursed_pattern = resolve_type(htype, patterns, recursed_types)
            resolved_words.append(word)
            recursed_patterns.extend(recursed_pattern)
        else:
            resolved_words.append(word)

    return resolved_words, recursed_patterns

def resolve_all_patterns(patterns):
    resolved_patterns_without_recursions = {}
    recursions = {}
    for name, pattern in patterns.items():
        resolved_pattern, recursed_patterns = resolve_pattern(name, pattern, patterns)
        resolved_patterns_without_recursions[name] = " ".join(resolved_pattern)
        if recursed_patterns:
            recursions[name] = sorted(recursed_patterns)
            
    print(resolved_patterns_without_recursions)
    resolved_patterns = dict(resolved_patterns_without_recursions)
    for name, recursion_list in recursions.items():
        if name in recursion_list:
            resolved_patterns[name] = f"{name}\n @ {name} {resolved_patterns_without_recursions[name]}"
        for recursion in recursion_list:
            if recursion != name:
                resolved_patterns[name] += f"\n @ {recursion} {resolved_patterns_without_recursions[recursion]}"

    return resolved_patterns

resolved_patterns = resolve_all_patterns(patterns)
resolved_patterns



{'rectangle': 'width:float mm x height:float mm', 'box': 'base:rectangle x depth:float mm', 'krabice': 'width:float mm x height:float mm x depth:float mm', 'choice': 'C weight:int || B foo:int || A bar:int', 'int_list': 'item:int rest_of_list:int_list || end of list', 'list': '<a> item:a rest_of_list:list<a> || end of list', 'double_recursion_a': 'drb:double_recursion_b dra:double_recursion_a', 'double_recursion_b': 'dra:double_recursion_a'}


{'rectangle': 'width:float mm x height:float mm',
 'box': 'base:rectangle x depth:float mm',
 'krabice': 'width:float mm x height:float mm x depth:float mm',
 'choice': 'C weight:int || B foo:int || A bar:int',
 'int_list': 'int_list\n @ int_list item:int rest_of_list:int_list || end of list',
 'list': '<a> item:a rest_of_list:list<a> || end of list',
 'double_recursion_a': 'double_recursion_a\n @ double_recursion_a drb:double_recursion_b dra:double_recursion_a',
 'double_recursion_b': 'double_recursion_b\n @ double_recursion_b dra:double_recursion_a\n @ double_recursion_a drb:double_recursion_b dra:double_recursion_a'}